# Score de Riesgo Bancario (Credit Scoring)

Notebook para calcular el score de riesgo crediticio con datos sintéticos o CSV externo.

## 1. Importar librerías

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, accuracy_score
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import optuna
from optuna.samplers import TPESampler
import os
import psutil

# Verificar que todas las librerías estén importadas correctamente
print("✅ Librerías importadas correctamente:")
print(f"NumPy: {np.__version__}")
print(f"Pandas: {pd.__version__}")
print(f"XGBoost: {xgb.__version__}")
print(f"Optuna: {optuna.__version__}")

# Verificar hardware disponible
print(f"\n🖥️ Hardware disponible:")
print(f"CPU Cores: {psutil.cpu_count(logical=False)} físicos, {psutil.cpu_count(logical=True)} lógicos")
print(f"RAM: {psutil.virtual_memory().total / (1024**3):.1f} GB")
print(f"RAM disponible: {psutil.virtual_memory().available / (1024**3):.1f} GB")

# Verificar soporte GPU para XGBoost
try:
    import subprocess
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
    if result.returncode == 0:
        print("🎮 GPU NVIDIA detectada - XGBoost puede usar GPU")
        GPU_AVAILABLE = True
    else:
        print("⚠️ GPU NVIDIA no detectada - usando CPU")
        GPU_AVAILABLE = False
except:
    print("⚠️ No se pudo verificar GPU - usando CPU")
    GPU_AVAILABLE = False

print("🚀 Listo para optimización bayesiana optimizada!")

## 2. Cargar datos
### A) Generar datos sintéticos

In [ ]:
np.random.seed(42)
# Generar dataset más grande aprovechando la RAM (64GB)
n = 10000  # 10x más datos para mejor entrenamiento
print(f"📊 Generando dataset de score de riesgo con {n:,} registros...")

data = pd.DataFrame({
    'historial_credito': np.random.randint(0, 10, n),  # Más rango
    'ingresos': np.random.lognormal(8, 0.5, n),       # Distribución log-normal
    'edad': np.random.randint(18, 80, n),             # Más rango de edad
    'relacion_banco': np.random.randint(1, 15, n),    # Más años de relación
    'segmento': np.random.choice(['Retail','Premium','Pyme','Empresarial'], n, p=[0.4, 0.3, 0.2, 0.1]),
    'deuda_total': np.random.lognormal(6, 1, n),      # Deuda total
    'productos_activos': np.random.randint(1, 8, n),  # Productos bancarios
    'empleo_estable': np.random.binomial(1, 0.7, n),  # Empleo estable
    'educacion': np.random.choice(['Basica','Media','Superior','Postgrado'], n, p=[0.2, 0.4, 0.3, 0.1]),
    'vivienda': np.random.choice(['Propia','Alquilada','Familiar'], n, p=[0.5, 0.3, 0.2]),
})

# Generar variable objetivo (riesgo) más realista
riesgo_prob = (
    0.25 * (data['historial_credito'] < 3) +              # Mal historial
    0.2 * (data['ingresos'] < 2000) +                     # Bajos ingresos
    0.15 * (data['edad'] < 25) +                          # Jóvenes
    0.1 * (data['deuda_total'] > data['ingresos'] * 0.5) + # Alto endeudamiento
    0.1 * (data['empleo_estable'] == 0) +                 # Empleo inestable
    0.1 * (data['productos_activos'] == 1) +              # Pocos productos
    0.05 * (data['relacion_banco'] < 2) +                 # Poca relación bancaria
    np.random.rand(n) * 0.05                              # Ruido aleatorio
)

data['riesgo'] = (riesgo_prob > 0.3).astype(int)
print(f"✅ Dataset generado: {data.shape[0]:,} filas x {data.shape[1]} columnas")
print(f"📈 Tasa de riesgo: {data['riesgo'].mean():.2%}")
data.head()

### B) Cargar datos desde CSV
> Descomenta y ajusta la ruta si tienes tu propio archivo.

In [ ]:
# data = pd.read_csv('clientes_score.csv')
# data.head()

## 3. Análisis exploratorio

In [ ]:
print(data['riesgo'].value_counts())
data.describe()
data.groupby('segmento')['riesgo'].mean().plot(kind='bar')
plt.title('Tasa de riesgo por segmento')
plt.ylabel('Riesgo')
plt.show()

## 4. Preprocesamiento

In [ ]:
X = data.drop('riesgo', axis=1)
y = data['riesgo']
X['segmento'] = LabelEncoder().fit_transform(X['segmento'])
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

## 5. Modelos
### A) Regresión Logística

In [ ]:
lr = LogisticRegression(max_iter=200)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print(classification_report(y_test, y_pred_lr))

### B) XGBoost

In [ ]:
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

## 6. Evaluación y visualización

In [ ]:
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plt.title('Confusión LR')
plt.imshow(confusion_matrix(y_test, y_pred_lr), cmap='Blues')
plt.subplot(1,2,2)
plt.title('Confusión XGBoost')
plt.imshow(confusion_matrix(y_test, y_pred_xgb), cmap='Greens')
plt.show()
fpr_lr, tpr_lr, _ = roc_curve(y_test, lr.predict_proba(X_test)[:,1])
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, xgb_model.predict_proba(X_test)[:,1])
plt.plot(fpr_lr, tpr_lr, label='LogisticReg')
plt.plot(fpr_xgb, tpr_xgb, label='XGBoost')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.legend()
plt.title('Curvas ROC')
plt.show()

## 7. Importancia de variables
### XGBoost

In [ ]:
xgb.plot_importance(xgb_model)
plt.show()

## 8. Comentarios finales
- Modifica parámetros, prueba con tu CSV y agrega visualizaciones adicionales.